<span>
<b>Author:</b> <a href="http://pages.di.unipi.it/ruggieri/">Salvatore Ruggieri</a><br/>
<b>Python version:</b>  3.x<br/>
</span>

In [1]:
try:
    # if using Colab
    import google.colab
    is_colab = True
    wdir = 'https://raw.githubusercontent.com/ruggieris/DD/main/'
    # create folder src if it does not exists
    !mkdir -p src
    # download source
    !wget --no-cache --backups=1 -O src/dd.py {wdir}'src/dd.py'
except:
    is_colab = False
    wdir = '../' # local files
print('Working dir: ', wdir)

Working dir:  ../


In [2]:
# required modules (under Anaconda use: > conda install -c conda-forge <package>)
if is_colab:
    !pip install pyroaring
    !pip install pyfim 
    !pip install lightgbm
    !pip install fairlearn

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('src/' if is_colab else '../src/') # local path
import dd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Part I: independence metrics

In [4]:
# Sample data: credit.csv, adult.csv or any other dataset with discrete columns only.
df = pd.read_csv(wdir+'data/credit_discrete.csv', sep=',', na_values='?')
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,lt_0,le_17d6,critical_or_other_existing_credit,radio_or_tv,le_38848d8,no_known_savings,ge_7,gt_2d8,male_single,none,...,real_estate,gt_52d6,none,own,from_1d6_le_2d2,skilled,le_1d2,yes,no,good
1,from_0_lt_200,gt_31d2,existing_paid,radio_or_tv,from_38848d8_le_7519d6,lt_100,from_1_lt_4,from_1d6_le_2d2,female_div_or_dep_or_mar,none,...,real_estate,le_30d2,none,own,le_1d6,skilled,le_1d2,none,no,bad
2,no_checking,le_17d6,critical_or_other_existing_credit,education,le_38848d8,lt_100,from_4_lt_7,from_1d6_le_2d2,male_single,none,...,real_estate,from_41d4_le_52d6,none,own,le_1d6,unskilled_resident,gt_1d2,none,no,good
3,lt_0,gt_31d2,existing_paid,furniture_or_equipment,from_7519d6_le_11154d4,lt_100,from_4_lt_7,from_1d6_le_2d2,male_single,guarantor,...,life_insurance,from_41d4_le_52d6,none,for_free,le_1d6,skilled,gt_1d2,none,no,good
4,lt_0,from_17d6_le_31d2,delayed_previously,new_car,from_38848d8_le_7519d6,lt_100,from_1_lt_4,gt_2d8,male_single,none,...,no_known_property,gt_52d6,none,for_free,from_1d6_le_2d2,skilled,gt_1d2,none,no,bad


In [5]:
# DD(filename or dataframe, unprotected item, negative decision)
#disc = dd.DD(df, 'foreign_worker=no', 'class=bad')
disc = dd.DD(df, unprotectedItem='age=from_41d4_le_52d6', predBadItem='class=bad')
for ctg in disc.ctg_global():
    disc.print(ctg)
    print("RD = {:f}".format(ctg.rd()))
ctg = disc.ctg_any()
disc.print(ctg)
print("RD = {:f}".format(ctg.rd()))

-----
Context ALL
Size = 1000  Perc = 100.00%
                     |class=bad|class=good|   
age=gt_52d6          |       29|        67| 96
age=from_41d4_le_52d6|       39|       122|161
                     |       68|       189|257
RD = 0.059847
-----
Context ALL
Size = 1000  Perc = 100.00%
                     |class=bad|class=good|   
age=le_30d2          |      148|       263|411
age=from_41d4_le_52d6|       39|       122|161
                     |      187|       385|572
RD = 0.117861
-----
Context ALL
Size = 1000  Perc = 100.00%
                     |class=bad|class=good|   
age=from_30d2_le_41d4|       84|       248|332
age=from_41d4_le_52d6|       39|       122|161
                     |      123|       370|493
RD = 0.010776
-----
Context ALL
Size = 1000  Perc = 100.00%
                      |class=bad|class=good|    
age!=from_41d4_le_52d6|      261|       578| 839
age=from_41d4_le_52d6 |       39|       122| 161
                      |      300|       700|1000
RD = 0.068849


In [6]:
# filtering condition: return None to filter out, or measure value
# contingency table ctg such that ctg.n() >= minSupp 
'''
     contingency table for independence
     =========== pred.pos === pred.neg === 
     protected       a            b       n1()
     unprotected     c            d       n2()
     ===========    m1()  ===    m2()  ==  n()
'''
def check_rd(ctg):
    # at least 20 protected with pred.pos and p2() > 0
    if ctg.a < 20 or ctg.c==0:
        return None
    return ctg.rd()

def check_rr(ctg):
    # at least 20 protected with pred.pos and p2() > 0
    if ctg.a < 20 or ctg.c==0:
        return None
    return ctg.rr()

In [7]:
# Extract contingency tables: 
# minSupp = min support of context (negative = absolute, positive = percentage)
# topk = top k contingency tables
ctgs_rd = disc.extract(testCond=check_rd, minSupp=-20, topk=1000)

In [8]:
# print top 3 
for v, ctg in ctgs_rd[:3]:
    disc.print(ctg)
    print("RD = {:f}".format(v))

-----
Context duration=gt_31d2 AND existing_credits=from_1d6_le_2d2 AND foreign_worker=no
Size = 60  Perc = 6.00%
                     |class=bad|class=good|  
age=le_30d2          |       21|         6|27
age=from_41d4_le_52d6|        1|         4| 5
                     |       22|        10|32
RD = 0.577778
-----
Context duration=gt_31d2 AND existing_credits=from_1d6_le_2d2
Size = 60  Perc = 6.00%
                     |class=bad|class=good|  
age=le_30d2          |       21|         6|27
age=from_41d4_le_52d6|        1|         4| 5
                     |       22|        10|32
RD = 0.577778
-----
Context purpose=new_car AND personal_status=female_div_or_dep_or_mar AND credit_amount=le_38848d8 AND foreign_worker=no
Size = 57  Perc = 5.70%
                     |class=bad|class=good|  
age=le_30d2          |       20|         8|28
age=from_41d4_le_52d6|        1|         4| 5
                     |       21|        12|33
RD = 0.514286


In [ ]:
# contingency tables wrt RR
ctgs_rr = disc.extract(testCond=check_rr, minSupp=-20, topk=1000)
for v, ctg in ctgs_rr[:3]:
    disc.print(ctg)
    print("RR = {:f}".format(v))

In [ ]:
# contingency tables in ctgs_rd
ct_rd_set = set(ctg[1] for ctg in ctgs_rd)
# contingency tables in ctgs_rd
ct_rr_set = set(ctg[1] for ctg in ctgs_rr)
# contingency tables in both
shared = ct_rd_set & ct_rr_set
len(shared) 

In [ ]:
# plot p1() vs p2()
plt.style.use('seaborn-whitegrid')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.plot([ctg.p1() for ctg in shared], [ctg.p2() for ctg in shared], '+', color='red', label=r'RR $\cap$ RD')
only_rd = ct_rd_set - shared
plt.plot([ctg.p1() for ctg in only_rd], [ctg.p2() for ctg in only_rd], '.', color='blue', label=r'RD \ RR')
only_rr = ct_rr_set - shared
plt.plot([ctg.p1() for ctg in only_rr], [ctg.p2() for ctg in only_rr], 'x', color='green', label=r'RR \ RD')
plt.legend()
plt.xlabel('p1')
plt.ylabel('p2')

In [ ]:
# sequential covering algorithm: 10 contingency tables
covers, residuals, times, uncovered, ctg, ctg_rem = disc.cover_n(ct_rd_set, check_rd, 10)
print('Total protected covered:', sum(residuals))
print('Total protected:', sum(residuals)+len(uncovered))
print('% covered: {:.2f}%'.format(100*sum(residuals)/(sum(residuals)+len(uncovered))))
disc.print(ctg)
print("RD = {:f}".format(ctg.rd()))
disc.print(ctg_rem)
print("RD = {:f}".format(ctg_rem.rd()))
for ctg, res in zip(covers, residuals):
    print('-----\nCovered', res)
    disc.print(ctg)
    print("RD = {:f}".format(ctg.rd()))

# Part II

In [ ]:
# Adult dataset
adult = pd.read_csv(wdir+'data/adult_discrete.csv', sep=',', na_values='?')
adult.head()

In [ ]:
# Encode categorical values
enc = dd.Encode()
df = enc.fit_transform(adult)
df.head()

In [ ]:
# there are missing values (but LightGBM deal with them! no need for imputation methods)
df.isna().sum()

In [ ]:
# df has categorical features (but LightGBM deal with them! no need for one-hot encoding)
df.dtypes

In [ ]:
# split train test
X = df[df.columns.drop('class')]
y = df['class'].astype(int)
X_train, X_test, y_train, y_test, adult_train, adult_test = train_test_split(X, y, adult.copy(), test_size=0.33, random_state=42)

In [ ]:
# training model and make predictions 
import lightgbm as lgb

clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
# add predicted class in the adult_test (decoding back)
adult_test['predicted'] = [enc.decode['class'][v] for v in clf.predict(X_test)]
adult_test['predicted'].astype('category')
# add predicted score in the adult_test
adult_test['score'] = clf.predict_proba(X_test)[:,1]
adult_test.head()

In [ ]:
# discrimination in overall test set
atts = list(set(adult_test.columns)-set(['score'])) # do not use score in contexts
disc = dd.DD(adult_test[atts], unprotectedItem='race=White', predBadItem='predicted=-50K', codes=dict(),
                                                 trueBadItem='class=-50K', na_values={'nan'}) 
for ctg in disc.ctg_global():
    disc.print(ctg)
    print("TNR White = {:f}".format(ctg.tnru()))
    print("TNR NonWhite = {:f}".format(ctg.tnrp()))
    print("EOP = {:f}".format(ctg.eop()))

In [ ]:
# filtering condition: return None to filter out, or measure value
# contingency table ctg such that ctg.n() >= minSupp 
'''
     contingency table for separation
          protected                                   unprotected
     ========== pred.pos == pred.neg ===      === pred.pos  == tpred.neg === 
     true.pos    TPp          FNp      Pp()          TPu          FNu        Pu()
     true.neg    FPp          TNp      Np()          FPu          TNu        Nu()
     ==========   a     =====  b  ===  n1()   ===      c    ====    d   ===  n2()
'''
def check_eop(ctg):
    # at least 20 protected with pred.pos and p2() > 0
    if ctg.Pu() < 20 or ctg.Pp() < 20 or ctg.Np()==0 or ctg.Nu()==0:
        return None
    return ctg.eop()

In [ ]:
# Extract contingency tables: 
ctgs_eop = disc.extract(testCond=check_eop, minSupp=-100, topk=1000)

In [ ]:
# output top 3
for v, ctg in ctgs_eop[:3]:
    disc.print(ctg)
    print("TNR White = {:f}".format(ctg.tnru()))
    print("TNR NonWhite = {:f}".format(ctg.tnrp()))
    print("EOP = {:f}".format(v))

In [ ]:
# sequential covering algorithm: 10 contingency tables
covers, residuals, times, uncovered, ctg, ctg_rem = disc.cover_n([ctg for _,ctg in ctgs_eop], check_eop, 10)
print('Total protected covered:', sum(residuals))
print('Total protected:', sum(residuals)+len(uncovered))
print('% covered: {:.2f}%'.format(100*sum(residuals)/(sum(residuals)+len(uncovered))))
disc.print(ctg)
print("EOP = {:f}".format(ctg.eop()))
disc.print(ctg_rem)
print("EOP = {:f}".format(ctg_rem.eop()))

for ctg, res in zip(covers, residuals):
    print('-----\nCovered', res)
    disc.print(ctg)
    print("TPR White = {:f}".format(ctg.tnru()))
    print("TPR NonWhite = {:f}".format(ctg.tnrp()))
    print("EOP = {:f}".format(ctg.eop()))

In [ ]:
# Fairlearn algorithms and utils (https://github.com/fairlearn/fairlearn)
# or try your preferred fair ML tool
from fairlearn.postprocessing import ThresholdOptimizer
# fairness by post-processing
postprocess_est = ThresholdOptimizer(estimator=clf, constraints="true_negative_rate_parity", prefit=True, predict_method='predict')
X_train = X_train.fillna(0) # fairlearn does not manage missing values
X_test = X_test.fillna(0) # fairlearn does not manage missing values
postprocess_est.fit(X_train, y_train, sensitive_features=X_train['race'])
# fair-corrected predictions 
adult_test['predicted'] = [enc.decode['class'][v] for v in postprocess_est.predict(X_test, sensitive_features=X_test['race']).astype(int)]
adult_test['predicted'].astype('category')

In [ ]:
# discrimination in test set
disc = dd.DD(adult_test[atts], unprotectedItem='race=White', predBadItem='predicted=-50K', codes=dict(),
                                                 trueBadItem='class=-50K', na_values={'nan'}) 
for ctg in disc.ctg_global():
    disc.print(ctg)
    print("TNR Male = {:f}".format(ctg.tnru()))
    print("TNR Female = {:f}".format(ctg.tnrp()))
    print("EOP = {:f}".format(ctg.eop()))

In [ ]:
# Extract contingency tables: 
ctgs_eop_post = disc.extract(testCond=check_eop, minSupp=-100, topk=1000)

In [ ]:
for v, ctg in ctgs_eop_post[:3]:
    disc.print(ctg)
    print("TNR White = {:f}".format(ctg.tnru()))
    print("TNR NonWhite = {:f}".format(ctg.tnrp()))
    print("EOP = {:f}".format(v))

In [ ]:
sns.set_style('whitegrid')
sns.kdeplot(np.array([v for v, _ in ctgs_eop]), bw_adjust=1.0, color='b')
sns.kdeplot(np.array([v for v, _ in ctgs_eop_post]), bw_adjust=1.0, color='r')
plt.legend(labels=['before', 'after'])

In [ ]:
# sequential covering algorithm: 10 contingency tables
covers, residuals, times, uncovered, ctg, ctg_rem = disc.cover_n([ctg for _,ctg in ctgs_eop_post], check_eop, 10)
print('Total protected covered:', sum(residuals))
print('Total protected:', sum(residuals)+len(uncovered))
print('% covered: {:.2f}%'.format(100*sum(residuals)/(sum(residuals)+len(uncovered))))
disc.print(ctg)
print("EOP = {:f}".format(ctg.eop()))
disc.print(ctg_rem)
print("EOP = {:f}".format(ctg_rem.eop()))

for ctg, res in zip(covers, residuals):
    print('-----\nCovered', res)
    disc.print(ctg)
    print("TNR White = {:f}".format(ctg.tnru()))
    print("TNR NonWhite = {:f}".format(ctg.tnrp()))
    print("EOP = {:f}".format(ctg.eop()))

# Part III

In [ ]:
# filtering condition: return None to filter out, or measure value
# contingency table ctg such that ctg.n() >= minSupp 
'''
     confusion matrix
     =========== pred.pos === pred.neg === 
     true.pos        a            b       n1()
     true.neg        c            d       n2()
     ===========    m1()  ===    m2()  ==  n()
'''
def check_err(ctg):
    return (ctg.b+ctg.c)/ctg.n()

In [ ]:
# errors in test set
disc = dd.DD(adult_test, unprotectedItem='class=-50K', predBadItem='predicted=+50K', codes=dict(), na_values={'nan'}) 
for ctg in disc.ctg_global():
    disc.print(ctg)
    print("ERR = {:f}".format(check_err(ctg)))

In [ ]:
# Extract contingency tables: 
ctgs_err = disc.extract(testCond=check_err, minSupp=-100, topk=1000)

In [ ]:
# Top 3
for v, ctg in ctgs_err[:3]:
    disc.print(ctg)
    print("ERR = {:f}".format(check_err(ctg)))

In [ ]:
def check_err2(ctg):
    n = ctg.n()
    return (ctg.b+ctg.c)/n if n>100 else None

covers, residuals, times, uncovered, ctg, ctg_rem = disc.cover_n([ctg for _,ctg in ctgs_err], check_err2, 10)
print('Total protected covered:', sum(residuals))
print('Total protected:', sum(residuals)+len(uncovered))
print('% covered: {:.2f}%'.format(100*sum(residuals)/(sum(residuals)+len(uncovered))))
disc.print(ctg)
print("ERR = {:f}".format(check_err(ctg)))
disc.print(ctg_rem)
print("ERR = {:f}".format(check_err(ctg_rem)))
for ctg, res in zip(covers, residuals):
    print(res)
    disc.print(ctg)
    print("ERR = {:f}".format(check_err(ctg)))

# Part IV

In [ ]:
# filtering condition: return None to filter out, or measure value
# contingency table ctg such that ctg.n() >= minSupp 
'''
     contingency table for inference (protected=None, n1=a,n2=c)
     ===========
     true.pos      a             
     true.neg      c             
     ===========   m1()
'''
def check_acc(ctg):
    n = ctg.m1()
    return max(ctg.a,ctg.c)/n if n>0 else None

In [ ]:
# accuracy in training set
disc = dd.DD(adult_train, 'class=-50K', codes=dict(), na_values={'nan'}) 
for ctg in disc.ctg_global():
    disc.print(ctg)
    print("ACC = {:f}".format(check_acc(ctg)))

In [ ]:
# Extract contingency tables: 
ctgs_acc = disc.extract(testCond=check_acc, minSupp=-1000, topk=1000)

In [ ]:
for v, ctg in ctgs_acc[:3]:
    disc.print(ctg)
    print("ACC = {:f}".format(check_acc(ctg)))

In [ ]:
# TBD: boundary kernel limiting to (0.0, 1.0)
sns.kdeplot(np.array([v for v, _ in ctgs_acc]), bw_adjust=1.0, color='b')